# Advanced Querying Mongo

Importing libraries and setting up connection

In [317]:
from pymongo import MongoClient
import pandas as pd
import re


In [318]:
str_conn='mongodb://localhost:27017'

cursor=MongoClient(str_conn)



In [319]:
#me aseguro que companies está en mongo
cursor.list_database_names()

['admin', 'companies', 'config', 'local']

In [320]:
#selecciono una base de datos
com = cursor.companies
#selecciono la coleccion
colec = com.companies

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [321]:
# Your Code
query = {'name':'Babelgum'}
select = {'_id': 0, 'name': 1} # 0 es que no venga, 1 es que venga, es para columnas
list(colec.find(query,select))


[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [322]:
# Your Code
query = {'number_of_employees': {'$gte': 5000}}
select = {'name': 1, 'number_of_employees': 1, '_id': 0}

results = colec.find(query, select).sort('number_of_employees', -1).limit(20)
df = pd.DataFrame(results)
df
#el -1 es ordenadar de forma desc y lo convierto a dataframe para verlo mejor

,name,number_of_employees
0,Siemens,405000
1,IBM,388000
2,Toyota,320000
3,PayPal,300000
4,Nippon Telegraph and Telephone Corporation,227000
5,Samsung Electronics,221726
6,Accenture,205000
7,Tata Consultancy Services,200300
8,Flextronics International,200000
9,Safeway,186000


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [323]:
# Your Code

query={'$and': [{'founded_year': {'$lt': 2005}},
                {'founded_year': {'$gt': 2000}}]}
select = {'name': 1, 'founded_year': 1, '_id': 0}

results = colec.find(query, select)
df = pd.DataFrame(results)


In [324]:
df.head()

,name,founded_year
0,Digg,2004
1,Facebook,2004
2,StumbleUpon,2002
3,Gizmoz,2003
4,Plaxo,2002


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [325]:
# Your Code



### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [326]:
# Your Code
query={'$and': [{'number_of_employees': {'$lt': 1000}},
                {'founded_year': {'$lt': 2005}}]}
select = {'name': 1, 'number_of_employees': 1, '_id': 0}

results = colec.find(query, select).sort('number_of_employees', -1).limit(10)
df = pd.DataFrame(results)
df

,name,number_of_employees
0,Infinera Corporation,974
1,NorthPoint Communications Group,948
2,888 Holdings,931
3,Forrester Research,903
4,SonicWALL,900
5,Webmetrics,900
6,Cornerstone OnDemand,881
7,Mozilla,800
8,Buongiorno,800
9,Yelp,800


### 6. All the companies that don't include the `partners` field.

In [327]:
# Your Code
query = {'partners': []}
select = {'name': 1,'_id': 0}

results = colec.find(query,select)

#pongo el select porque el resto de columnas no me interesan quiero solo el nombre


df = pd.DataFrame(results)
df.head()

,name
0,Wetpaint
1,AdventNet
2,Zoho
3,Digg
4,Facebook


### 7. All the companies that have a null type of value on the `category_code` field.

In [328]:
# Your Code
query = {'category_code': None}
select = {'name': 1,'_id': 0}

results = colec.find(query, select)

df = pd.DataFrame(results)
df.head()

,name
0,Collective
1,Snimmer
2,KoolIM
3,Level9 Media
4,VidKing


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [329]:
# Your Code
query={'$and': [{'number_of_employees': {'$gte': 100}},
                {'number_of_employees': {'$lt': 1000}}]}
select = {'name': 1, 'number_of_employees': 1, '_id': 0}

results = colec.find(query, select)
df = pd.DataFrame(results)
df.head()

,name,number_of_employees
0,AdventNet,600
1,AddThis,120
2,OpenX,305
3,LifeLock,644
4,Jajah,110


### 9. Order all the companies by their IPO price in a descending order.

In [330]:
# Your Code

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [331]:
# Your Code

#lo he intenado directamente con un select pero hace falta un query así que 
#pongo en aquellos que exista un valor


query = {'number_of_employees': {'$exists': True}}
select = {'name': 1, 'number_of_employees': 1, '_id': 0}


results = colec.find(query,select).sort('number_of_employees', -1).limit(10)
df = pd.DataFrame(results)
df.head()


,name,number_of_employees
0,Siemens,405000
1,IBM,388000
2,Toyota,320000
3,PayPal,300000
4,Nippon Telegraph and Telephone Corporation,227000


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [332]:
# Your Code
#si el segundo semestre es desde el 1 de julio hasta el 31 de diciembre

query = {'founded_month': {'$gte': 7}}
select = {'name': 1, '_id': 0}

results = colec.find(query, select).limit(1000)
df = pd.DataFrame(results)
df.head()


,name
0,Wetpaint
1,Zoho
2,Digg
3,Omnidrive
4,eBay


### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [333]:
# aprovecho la función de Yona

def exchange(col):
    
    res=[]
    
    number=1
    
    symbol=1
    
    for e in col:
        
        if e[-1]=='k': number=1e3
        elif e[-1]=='M': number=1e6
        elif e[-1]=='B': number=1e9
            
        if e[0]=='£': symbol=1.3
        elif e[0]=='€' : symbol=1.12
        
        n_col=re.findall('[0-9.]+', e)[0]

        
        res.append(int(float(n_col) * number * symbol))
        
    return res

In [334]:
#primero hago la query que me interesa
query = {'founded_year': {'$lt': 2000}}
select = {'name': 1, '_id': 0,'total_money_raised':1}

results = colec.find(query, select)
data = pd.DataFrame(results)
data.head()

,name,total_money_raised
0,AdventNet,$0
1,Postini,$0
2,Fox Interactive Media,$0
3,eBay,$6.7M
4,CBS,$0


In [335]:
#ahora le aplico a la función
data['n_total_money']=exchange(data.total_money_raised)

data.head()

,name,total_money_raised,n_total_money
0,AdventNet,$0,0
1,Postini,$0,0
2,Fox Interactive Media,$0,0
3,eBay,$6.7M,6700000
4,CBS,$0,0


In [336]:
#ahora la condición
data_filtrado = data[data['n_total_money']>10000]
data_filtrado.head()

,name,total_money_raised,n_total_money
3,eBay,$6.7M,6700000
6,Cisco,$2.5M,2500000
7,Yahoo!,$6.8M,6800000
8,Google,$555M,555000000
10,SideStep,$32M,32000000


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [337]:
acquisitions = colec.distinct("acquisitions")
df_acquisitions = pd.DataFrame(acquisitions)
df_acquisitions.iloc[5]

price_amount                                                         NaN
price_currency_code                                                  EUR
term_code                                                           None
source_url             http://www.computing.co.uk/ctg/news/2273233/sa...
source_description     SAP to acquire Swiss e-commerce firm Hybris in...
acquired_year                                                     2013.0
acquired_month                                                       6.0
acquired_day                                                         6.0
company                        {'name': 'hybris', 'permalink': 'hybris'}
Name: 5, dtype: object

In [338]:
query = {'acquisitions': {'$exists': True}}
select = {'name': 1, '_id': 0,'acquisitions':1}

results = colec.find(query, select)
data = pd.DataFrame(results)
data.head()

,name,acquisitions
0,Wetpaint,[]
1,AdventNet,[]
2,Zoho,[]
3,Digg,"[{'price_amount': None, 'price_currency_code':..."
4,Facebook,"[{'price_amount': None, 'price_currency_code':..."


In [339]:
'''
def year_ac(year_acq):
    if year_acq:
        return year_acq.get("acquired_year")
    else:
        return None

data["acq_year"] = data["acquisitions"].apply(year_ac)
'''

#el problema es que acquisitions es una lista de un diccionario
#aquí obtengo el primer elemento de la lista que es el unico dic
#y saco el año 

def year_acq(lst):
    if lst:
        dct = lst[0]  
        return dct.get("acquired_year")
    else:
        return None

data["acq_year"] = data["acquisitions"].apply(year_acq)

def price(lst):
    if lst:
        dct = lst[0]  
        return dct.get("price_amount")
    else:
        return None

data["price"] = data["acquisitions"].apply(price)

data



,name,acquisitions,acq_year,price
0,Wetpaint,[],NaN,NaN
1,AdventNet,[],NaN,NaN
2,Zoho,[],NaN,NaN
3,Digg,"[{'price_amount': None, 'price_currency_code':...",2009.0,NaN
4,Facebook,"[{'price_amount': None, 'price_currency_code':...",2007.0,NaN
...,...,...,...,...
18796,Adhunk,[],NaN,NaN
18797,AfterLogic,[],NaN,NaN
18798,goBookmaker,[],NaN,NaN
18799,EnteGreat Solutions,"[{'price_amount': 5120000, 'price_currency_cod...",2008.0,5120000.0


In [340]:
#lo convierto a int pero antes me cargo los nulos
data = data.dropna(subset=['acq_year'])
data = data.dropna(subset=['price'])
data['acq_year'] = data['acq_year'].astype(int)
data['price'] = data['price'].astype(int)

data


,name,acquisitions,acq_year,price
9,Fox Interactive Media,"[{'price_amount': 20000000, 'price_currency_co...",2007,20000000
10,Twitter,"[{'price_amount': 15000000, 'price_currency_co...",2008,15000000
11,StumbleUpon,"[{'price_amount': 29000000, 'price_currency_co...",2009,29000000
17,eBay,"[{'price_amount': 1500000000, 'price_currency_...",2002,1500000000
20,CBS,"[{'price_amount': 280000000, 'price_currency_c...",2007,280000000
...,...,...,...,...
18749,Emrise Corporation,"[{'price_amount': 18780000, 'price_currency_co...",2009,18780000
18789,Axon Solutions,"[{'price_amount': 17500000, 'price_currency_co...",2009,17500000
18791,Willdan Group,"[{'price_amount': 16460000, 'price_currency_co...",2009,16460000
18793,QSGI,"[{'price_amount': 15040000, 'price_currency_co...",2008,15040000


In [341]:
data_filtrado = data[data['acq_year'] > 2008].sort_values('price', ascending=False)


In [342]:
data_filtrado.head()

,name,acquisitions,acq_year,price
11174,Bristol-Myers Squibb,"[{'price_amount': 2100000000, 'price_currency_...",2009,2100000000
12007,Visa,"[{'price_amount': 2000000000, 'price_currency_...",2010,2000000000
1441,Open Source Food,"[{'price_amount': 1400000000, 'price_currency_...",2010,1400000000
12597,Cablevision Systems,"[{'price_amount': 1360000000, 'price_currency_...",2010,1360000000
13803,Schlumberger,"[{'price_amount': 1070000000, 'price_currency_...",2010,1070000000


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [343]:
# Your Code

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [344]:
# Your Code

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [345]:
# Your Code

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [346]:
# Your Code

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [347]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [348]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [349]:
# Your Code